In [1]:
import pandas as pd
import os
import numpy as np
import heapq
import editdistance as Levdist
from pyxdameraulevenshtein import damerau_levenshtein_distance as DLevdist
from pyxdameraulevenshtein import normalized_damerau_levenshtein_distance as NDLevdist

In [2]:
# actdats - start date
# cname - company name
# analyst - analyst
# baname - analyst
# estimid - bank

In [3]:
recdet = pd.read_stata('../data/recdet.dta')
bran = pd.read_stata('../data/bran.dta')

In [4]:
merged = pd.merge(recdet,
                 bran,
                 how='outer',
                 left_on=['amaskcd'],
                 right_on=['bacode'])

In [5]:
merged.head()

,ticker,cusip,cname,oftic,actdats,estimid,analyst,ereccd,etext,ireccd,...,acttims,revdats,revtims,anndats,anntims,baindi,bacode,baname,baid,usfirm_y
0,0000,87482X10,TALMER BANCORP,TLMR,2014-03-10,RBCDOMIN,ARFSTROM J,2,OUTPERFORM,2,...,08:54:03,2016-01-26,09:35:52,2014-03-10,00:20:00,A,71182.0,ARFSTROM J,,1.0
1,0000,87482X10,TALMER BANCORP,TLMR,2016-01-26,RBCDOMIN,ARFSTROM J,3,SECTOR PERFORM,3,...,21:57:09,2016-04-28,10:48:13,2016-01-26,21:44:00,A,71182.0,ARFSTROM J,,1.0
2,0097,39260X10,GREEN BANCORP,GNBC,2014-09-02,RBCDOMIN,ARFSTROM J,2,OUTPERFORM,2,...,08:13:22,2016-07-28,21:40:22,2014-09-02,07:50:00,A,71182.0,ARFSTROM J,,1.0
3,AINV,03761U10,APOLLO INVESTMEN,AINV,2009-08-08,RBCDOMIN,ARFSTROM J,2,OUTPERFORM,2,...,18:58:54,2017-05-18,09:22:18,2009-08-07,06:53:00,A,71182.0,ARFSTROM J,,1.0
4,ALLE,01747610,ALLEGIANT BANC,ALLE,2002-01-14,RBCDOMIN,ARFSTROM J,2,OUTPERFORM,2,...,16:22:51,2003-10-27,15:08:44,2002-01-14,16:16:21,A,71182.0,ARFSTROM J,,1.0


In [6]:
def sort_by_levenstein(list1, list2, n_sim_cols, l1_name='list1', l2_name='list2'):
#     if len(list1)-len(list2)>=0:
    listb = list1
    listm = list2
    cols = [l1_name]+[l2_name+'_'+str(nsim+1) for nsim in range(n_sim_cols)]
#     else:
#         listb = list2
#         listm = list1
#         cols = [l2_name]+[l1_name+'_'+str(nsim+1) for nsim in range(n_sim_cols)]
    
    result = []
    similar = []
    for l1 in range(len(listb)):
#         if len(listm)!=1:
        cur_lev_dist = []
        for l2 in range(len(listm)):
            w_distance = DLevdist(listb[l1], listm[l2])
            cur_lev_dist.append(w_distance)
            if w_distance==0:
                similar.append([listb[l1], listm[l2]])

        min_indexes = heapq.nsmallest(n_sim_cols, range(len(cur_lev_dist)), cur_lev_dist.__getitem__)
        result.append([listb[l1]]+[listm[min_indexes[nsim]] for nsim in range(n_sim_cols)])
#             listm.pop(min_indexes[0])
#         else:
#             result.append([listb[l1], np.nan, np.nan])
    return pd.DataFrame(result, columns=cols), pd.DataFrame(similar, columns=cols[:2])

In [7]:
def preproc_names_list(l):
    l = list(map(lambda x: x.replace(' ', ''), l))
    l = list(map(lambda x: x.replace('.', ''), l))
    l = list(map(lambda x: x.replace(',', ''), l))
    l = list(map(lambda x: x.replace('&', ''), l))
    l = list(map(lambda x: x.replace('-', ''), l))
    l = list(map(lambda x: x.replace(')', ''), l))
    l = list(map(lambda x: x.replace('(', ''), l))
    l = list(map(lambda x: x.replace(']', ''), l))
    l = list(map(lambda x: x.replace('[', ''), l))
    l = list(map(lambda x: x.replace('/', ''), l))
    l = list(map(lambda x: x.replace(':', ''), l))
    l = list(map(lambda x: x.replace(';', ''), l))
    l = list(map(lambda x: x.replace('$', '').lower(), l))
    l = list(map(lambda x: x.replace('company', ''), l))
    l = list(map(lambda x: x.replace('research', ''), l))
    if 'no_company_found' in l:
        l.remove('no_company_found')
    if '' in l:
        l.remove('')
    return pd.unique(l).tolist()

In [13]:
recdet_banks = pd.unique(merged['cname'].dropna().tolist())

In [14]:
recdet_banks = preproc_names_list(recdet_banks)

In [17]:
recdet_banks

['talmerbancorp',
 'greenbancorp',
 'apolloinvestmen',
 'allegiantbanc',
 'assocbanccpwi',
 'bancorpsouthinc',
 'bokfinancialcp',
 '1stbksamerica',
 'cathaygeneral',
 'citizensrepubli',
 'concordefs',
 'cullenfrosttx',
 'cmnty1stbcsh',
 'capitolfederal',
 'skyfinancialgr',
 'comericaincmi',
 'metrobancorp',
 'compassdiversif',
 'firstmeritcp',
 'usbancorp',
 'franklinbankco',
 '1stdata',
 '1stfinlbncp',
 'fiservinc',
 '5th3rdbcpoh',
 'firstarnew',
 'firstarcorp',
 'usbancorpnew',
 '1stmerchantscp',
 'fifthstreetfin',
 'firsthorizon',
 'goldbanccp',
 'grtsouthnbncp',
 'greatwestern',
 'huntingtbcshoh',
 'hancockhldgco',
 'homebancshares',
 'herculestechnol',
 'heartlandfinl',
 'independntbkmi',
 'investorsfincl',
 'mercantilebank',
 'macatawabank',
 'multexcom',
 'marshilsleywi',
 'southwestbcpok',
 'oldnatlbancin',
 'bankonecpoh',
 'patriotcapital',
 'prosperitybksh',
 'prospectcapital',
 'privatebancorp',
 'republicbancorp',
 'republic1stbcp',
 'sterlingbancsha',
 'southwestbanc',
 't

---

#### NEW PART 1

In [9]:
treshold = 10

In [10]:
banks_freq = pd.read_excel("banks_freq.xlsx")

In [11]:
banks_freq_comps = banks_freq[banks_freq["freq"]>=treshold].index.dropna().tolist()

---

In [12]:
sa_banks = preproc_names_list(banks_freq_comps)

In [13]:
recdet_banks = preproc_names_list(recdet_banks)

In [14]:
res, similar = sort_by_levenstein(sa_banks, recdet_banks, 10, "SA", "RECDET")

In [15]:
len(res), len(similar)

(1418, 61)

---

#### NEW PART 2

In [16]:
links = pd.read_excel("banks_linking_first.xlsx")

In [17]:
index_links = links[["RECDET_link1", "RECDET_link2", "RECDET_link3"]].dropna(how="all").index

In [18]:
links_done = links.loc[index_links, :].copy()

In [19]:
banks_freq_top = banks_freq[banks_freq["freq"]>=treshold].copy()
banks_freq_top = banks_freq_top.loc[banks_freq_top.index.dropna(),:].copy()
banks_freq_top["bank"] = banks_freq_top.index
banks_freq_top["bank_processed"] = banks_freq_top.apply(lambda x: preproc_names_list([x["bank"]])[0], axis=1)

In [20]:
res_linking = pd.merge(
    banks_freq_top.sort_values("bank_processed")[["freq", "bank_processed"]],
    res,
    left_on="bank_processed",
    right_on="SA",
    how="right"
)

In [21]:
res_linking_final = pd.merge(
    res_linking,
    links_done,
    left_on="SA",
    right_on="SA",
    how="left"
)

In [22]:
res_linking_final.to_excel("links.xlsx")

---

In [98]:
res.to_excel('../banks_link.xlsx')

In [99]:
resSA, similarSA = sort_by_levenstein(sa_banks, sa_banks, 25, "SA1", "SA2")

In [100]:
resSA.to_excel('../SA_link.xlsx')

In [12]:
resRD, similarRD = sort_by_levenstein(recdet_banks, recdet_banks, 25, "RD1", "RD2")

In [14]:
resRD.to_excel("rd.xlsx")

In [14]:
pd.DataFrame(recdet_banks).to_excel("recdet_names.xlsx")